In [1]:
import polars as pl

In [3]:
data = pl.read_csv('amz_uk_price_prediction_dataset.csv')
df = data.clone()
df.head(5)

uid,asin,title,stars,reviews,price,isBestSeller,boughtInLastMonth,category
i64,str,str,f64,i64,f64,bool,i64,str
1,"""B09B96TG33""","""Echo Dot (5th generation, 2022…",4.7,15308,21.99,false,0,"""Hi-Fi Speakers"""
2,"""B01HTH3C8S""","""Anker Soundcore mini, Super-Po…",4.7,98099,23.99,true,0,"""Hi-Fi Speakers"""
3,"""B09B8YWXDF""","""Echo Dot (5th generation, 2022…",4.7,15308,21.99,false,0,"""Hi-Fi Speakers"""
4,"""B09B8T5VGV""","""Echo Dot with clock (5th gener…",4.7,7205,31.99,false,0,"""Hi-Fi Speakers"""
5,"""B09WX6QD65""","""Introducing Echo Pop | Full so…",4.6,1881,17.99,false,0,"""Hi-Fi Speakers"""
